In [10]:
# importing all the required libraries
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np


In [1]:
!!pip install kaggle

['Requirement already satisfied: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)',
 'Requirement already satisfied: certifi in /usr/local/lib/python3.7/dist-packages (from kaggle) (2020.12.5)',
 'Requirement already satisfied: urllib3 in /usr/local/lib/python3.7/dist-packages (from kaggle) (1.24.3)',
 'Requirement already satisfied: python-slugify in /usr/local/lib/python3.7/dist-packages (from kaggle) (4.0.1)',
 'Requirement already satisfied: six>=1.10 in /usr/local/lib/python3.7/dist-packages (from kaggle) (1.15.0)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.7/dist-packages (from kaggle) (4.41.1)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from kaggle) (2.23.0)',
 'Requirement already satisfied: python-dateutil in /usr/local/lib/python3.7/dist-packages (from kaggle) (2.8.1)',
 'Requirement already satisfied: text-unidecode>=1.3 in /usr/local/lib/python3.7/dist-packages (from python-slugify->kaggle) (1.3)',
 '

In [ ]:
from google.colab import files
files.upload()

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d alxmamaev/flowers-recognition

 95% 429M/450M [00:04<00:00, 83.7MB/s]
100% 450M/450M [00:04<00:00, 104MB/s] 


In [7]:
!unzip flowers-recognition.zip

Streaming output truncated to the last 5000 lines.
  inflating: flowers/flowers/rose/110472418_87b6a3aa98_m.jpg  
  inflating: flowers/flowers/rose/11102341464_508d558dfc_n.jpg  
  inflating: flowers/flowers/rose/11233672494_d8bf0a3dbf_n.jpg  
  inflating: flowers/flowers/rose/11694025703_9a906fedc1_n.jpg  
  inflating: flowers/flowers/rose/118974357_0faa23cce9_n.jpg  
  inflating: flowers/flowers/rose/11944957684_2cc806276e.jpg  
  inflating: flowers/flowers/rose/12165480946_c4a3fe182d_n.jpg  
  inflating: flowers/flowers/rose/12202373204_34fb07205b.jpg  
  inflating: flowers/flowers/rose/12238827553_cf427bfd51_n.jpg  
  inflating: flowers/flowers/rose/12240165555_98625b1e88_n.jpg  
  inflating: flowers/flowers/rose/12240303_80d87f77a3_n.jpg  
  inflating: flowers/flowers/rose/12240577184_b0de0e53ea_n.jpg  
  inflating: flowers/flowers/rose/12243068283_ee4c2683e2_n.jpg  
  inflating: flowers/flowers/rose/12243069253_e512464095_n.jpg  
  inflating: flowers/flowers/rose/123128873_546b8b

In [8]:
# data pre-processing
#install library
!pip install split_folders

In [9]:
import splitfolders

input_folder = "/content/flowers"
output = "/content/processed_dataset"

splitfolders.ratio(input_folder, output, seed=42, ratio=(.6, .2, .2))

Copying files: 4326 files [00:00, 4650.01 files/s]


In [11]:
img_H, img_W = (224,224) # standardizing the image dimensions
batch_size = 32


In [15]:

# data paths
train_dir = 'processed_dataset/train'
test_dir = 'processed_dataset/test'
valid_dir = 'processed_dataset/val'

In [16]:

# data generators
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    validation_split=0.4)

In [17]:

train_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(img_H, img_W),
    batch_size =batch_size,
    class_mode='categorical',
    subset='training')

valid_gen = train_data_gen.flow_from_directory(
    valid_dir,
    target_size=(img_H, img_H),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

test_gen = train_data_gen.flow_from_directory(
    test_dir,
    target_size=(img_H, img_W),
    batch_size=1,
    class_mode='categorical',
    subset='validation')


Found 1557 images belonging to 6 classes.
Found 342 images belonging to 6 classes.
Found 347 images belonging to 6 classes.


In [18]:
x, y = test_gen.next()
print(x.shape)


(1, 224, 224, 3)


In [19]:

# defining the model
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
pred = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=pred)

for layer in base_model.layers:
    layer.trainable=False

94773248/94765736 [==============================] - 0s 0us/step


In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, epochs=50)


Epoch 1/50
49/49 [==============================] - 22s 396ms/step - loss: 0.3692 - accuracy: 0.8829
Epoch 2/50
49/49 [==============================] - 19s 393ms/step - loss: 0.1972 - accuracy: 0.9227
Epoch 3/50
49/49 [==============================] - 19s 390ms/step - loss: 0.1457 - accuracy: 0.9456
Epoch 4/50
49/49 [==============================] - 19s 395ms/step - loss: 0.1601 - accuracy: 0.9510
Epoch 5/50
49/49 [==============================] - 19s 390ms/step - loss: 0.0628 - accuracy: 0.9822
Epoch 6/50
49/49 [==============================] - 19s 392ms/step - loss: 0.0687 - accuracy: 0.9751
Epoch 7/50
49/49 [==============================] - 19s 392ms/step - loss: 0.0498 - accuracy: 0.9883
Epoch 8/50
49/49 [==============================] - 19s 393ms/step - loss: 0.0652 - accuracy: 0.9823
Epoch 9/50
49/49 [==============================] - 19s 395ms/step - loss: 0.0331 - accuracy: 0.9873
Epoch 10/50
49/49 [==============================] - 19s 395ms/step - loss: 0.0370 - accura

In [22]:
# saving the model
model.save('/content/flowers_final_model.h5')

In [23]:
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print("\nTest accuracy: ", test_acc)

347/347 - 7s - loss: 0.7915 - accuracy: 0.8905

Test accuracy:  0.890489935874939


In [37]:
#testing the model 
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/sample_test_image.jpg')
	# load model
	model = load_model('flowers_final_model.h5')
	# predict the class
	result = model.predict(img)
	print("The image is of ",np.argmax(result))

# entry point, run the example
run_example()

The image is of  0
